# C2HO Hackathon - MCTeam

Creates the audio/visual submission from the MCT. For more details see the readme file. After the audio and visual elements are finished the programme turns into a game of pong for 2 players. Player 1's controls are the up and down arrow keys, Player 2's controls are the w and s keys. Esc key quits the programme.

## Build and run the logo

In [ ]:
import numpy as np
import pygame
from pygame.locals import *
import sys
import random
import pyttsx3
import socket
import librosa
import scipy
from scipy import signal
import skvideo.io
import pyaudio
from multiprocessing.pool import ThreadPool
import ffmpegio
import ffmpeg

In [ ]:
# Ip stuff

# Get IP
hostname=socket.gethostname()
IPAddr=socket.gethostbyname(hostname)

# Getting the IP as integers
ip = []
for i in IPAddr:
    try:
        ip.append(int(i))
    except ValueError:
        pass

# Extracting the name from the computer name

name = hostname.split(sep='-')

# check if windows or macos

if sys.platform == 'win32':
    name = name[0]
    
elif sys.platform == 'darwin':
    name = name[0]
    name = name[0:len(name)-1]

In [ ]:
# set pygame display variables

import random

def background_color_scheme(ip):
    random.seed(a=ip[-1], version=2)
    x = random.randint(0,255)
    y = random.randint(0,255)
    z = random.randint(0,255)
    return x,y,z

x,y,z = background_color_scheme(ip)

screen_fill = x, y, z # background colour - use ip digits
width, height = 640, 480

white = 255, 255, 255

fps = 60

# set audio variables

sr = 48000

In [ ]:
# classes for objects in pygame

class Square():
    
    def __init__(self, colour, pos_x, pos_y, size):
        self.colour = colour
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.size = size

    def draw(self, screen):

        pygame.draw.rect(screen, self.colour, (self.pos_x, self.pos_y, self.size, self.size))
        
class Paddle():
    
    def __init__(self, colour, pos_x, pos_y, width, height):
        self.colour = colour
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.width = width
        self.height = height
        
        
    def draw(self, screen):
        
        rect = pygame.draw.rect(screen, self.colour, (self.pos_x, self.pos_y, self.width, self.height))
        
        return rect

In [ ]:
# Functions for drawing C, 2, H, O

def drawH(centre_x, centre_y, dim_x, dim_y, square_size):
    
    while int(dim_x*0.6)%square_size != 0 or int(dim_y*0.4)%square_size != 0:
        square_size += 1        
    
    square_list = []
    
    # left bar
    
    l_bar_length = int(dim_x*0.2)
    l_bar_height = int(dim_y)
    l_init_x = centre_x-(dim_x/2)
    l_init_y = centre_y-(dim_y/2)
    l_pos_x = np.arange(l_init_x, l_init_x+l_bar_length, square_size)
    l_pos_y = np.arange(l_init_y, l_init_y+l_bar_height, square_size)
    l_pos_grid = np.meshgrid(l_pos_x, l_pos_y)
    l_pos_xx = l_pos_grid[0].flatten()
    l_pos_yy = l_pos_grid[1].flatten()
    
    
    for i in range(len(l_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), l_pos_xx[i], l_pos_yy[i], square_size))
    
    # centre bar
    
    c_bar_length = int(dim_x*0.6)
    c_bar_height = int(dim_y*0.4)
    c_init_x = l_pos_x.max()+square_size
    c_init_y = centre_y-(c_bar_height/2)
    c_pos_x = np.arange(c_init_x, c_init_x+c_bar_length, square_size)
    c_pos_y = np.arange(c_init_y, c_init_y+c_bar_height, square_size)
    c_pos_grid = np.meshgrid(c_pos_x, c_pos_y)
    c_pos_xx = c_pos_grid[0].flatten()
    c_pos_yy = c_pos_grid[1].flatten()
    
    
    for i in range(len(c_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), c_pos_xx[i], c_pos_yy[i], square_size))
    
        
    # right bar
    
    r_bar_length = int(dim_x*0.2)
    r_bar_height = int(dim_y)
    r_init_x = c_pos_x.max()+square_size
    r_init_y = centre_y-(dim_y/2)
    r_pos_x = np.arange(r_init_x, r_init_x+r_bar_length, square_size)
    r_pos_y = np.arange(r_init_y, r_init_y+r_bar_height, square_size)
    r_pos_grid = np.meshgrid(r_pos_x, r_pos_y)
    r_pos_xx = r_pos_grid[0].flatten()
    r_pos_yy = r_pos_grid[1].flatten()
    
    
    for i in range(len(r_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), r_pos_xx[i], r_pos_yy[i], square_size))
    

        
    return square_list

def drawC(centre_x, centre_y, dim_x, dim_y, square_size):
    
    while int(dim_x*0.6)%square_size != 0 or int(dim_y*0.4)%square_size != 0:
        square_size += 1        
    
    square_list = []
    
    # top bar
    
    t_bar_length = dim_x
    t_bar_height = int(dim_y*0.2)
    t_init_x = centre_x-(dim_x/2)
    t_init_y = centre_y-(dim_y/2)+(dim_y*0.2)
    t_pos_x = np.arange(t_init_x, t_init_x+t_bar_length, square_size)
    t_pos_y = np.arange(t_init_y, t_init_y+t_bar_height, square_size)
    t_pos_grid = np.meshgrid(t_pos_x, t_pos_y)
    t_pos_xx = t_pos_grid[0].flatten()
    t_pos_yy = t_pos_grid[1].flatten()
    
    for i in range(len(t_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), t_pos_xx[i], t_pos_yy[i], square_size))
    
    # left bar
    
    l_bar_length = int(dim_x*0.2)
    l_bar_height = int(dim_y*0.6)
    l_init_x = t_init_x
    l_init_y = t_pos_y.max() + square_size
    l_pos_x = np.arange(l_init_x, l_init_x+l_bar_length, square_size)
    l_pos_y = np.arange(l_init_y, l_init_y+l_bar_height, square_size)
    l_pos_grid = np.meshgrid(l_pos_x, l_pos_y)
    l_pos_xx = l_pos_grid[0].flatten()
    l_pos_yy = l_pos_grid[1].flatten()
    
    for i in range(len(l_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), l_pos_xx[i], l_pos_yy[i], square_size))
        
    # bottom bar
    
    b_bar_length = dim_x
    b_bar_height = int(dim_y*0.2)
    b_init_x = centre_x-(dim_x/2)
    b_init_y = l_pos_y.max() + square_size
    b_pos_x = np.arange(b_init_x, b_init_x+b_bar_length, square_size)
    b_pos_y = np.arange(b_init_y, b_init_y+b_bar_height, square_size)
    b_pos_grid = np.meshgrid(b_pos_x, b_pos_y)
    b_pos_xx = b_pos_grid[0].flatten()
    b_pos_yy = b_pos_grid[1].flatten()
    
    for i in range(len(b_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), b_pos_xx[i], b_pos_yy[i], square_size))
    

        
    return square_list

def draw2(centre_x, centre_y, dim_x, dim_y, square_size):
    
    while int(dim_x*0.6)%square_size != 0 or int(dim_y*0.4)%square_size != 0:
        square_size += 1
        
    square_list = []
    
    # top bar
    
    t_bar_length = (dim_x*0.8)
    t_bar_height = int(dim_y*0.2)
    t_init_x = centre_x-(dim_x/2)
    t_init_y = centre_y-(dim_y/2)+(dim_y*0.2)
    t_pos_x = np.arange(t_init_x, t_init_x+t_bar_length, square_size)
    t_pos_y = np.arange(t_init_y, t_init_y+t_bar_height, square_size)
    t_pos_grid = np.meshgrid(t_pos_x, t_pos_y)
    t_pos_xx = t_pos_grid[0].flatten()
    t_pos_yy = t_pos_grid[1].flatten()
    
    for i in range(len(t_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), t_pos_xx[i], t_pos_yy[i], square_size))
        
    # right bar
    
    r_bar_length = int(dim_x*0.2)
    r_bar_height = int(dim_y*0.4)
    r_init_x = t_pos_x.max() + square_size
    r_init_y = t_pos_y.min()
    r_pos_x = np.arange(r_init_x, r_init_x+r_bar_length, square_size)
    r_pos_y = np.arange(r_init_y, r_init_y+r_bar_height, square_size)
    r_pos_grid = np.meshgrid(r_pos_x, r_pos_y)
    r_pos_xx = r_pos_grid[0].flatten()
    r_pos_yy = r_pos_grid[1].flatten()
    
    for i in range(len(r_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), r_pos_xx[i], r_pos_yy[i], square_size))
        
    # centre bar
    
    c_bar_length = int(dim_x)
    c_bar_height = int(dim_y*0.2)
    c_init_x = centre_x-(dim_x/2)
    c_init_y = r_pos_y.max() + square_size
    c_pos_x = np.arange(c_init_x, c_init_x+c_bar_length, square_size)
    c_pos_y = np.arange(c_init_y, c_init_y+c_bar_height, square_size)
    c_pos_grid = np.meshgrid(c_pos_x, c_pos_y)
    c_pos_xx = c_pos_grid[0].flatten()
    c_pos_yy = c_pos_grid[1].flatten()
    
    for i in range(len(c_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), c_pos_xx[i], c_pos_yy[i], square_size))
    
    # left bar
    
    l_bar_length = int(dim_x*0.2)
    l_bar_height = int(dim_y*0.2)
    l_init_x = centre_x-(dim_x/2)
    l_init_y = c_pos_y.max() + square_size
    l_pos_x = np.arange(l_init_x, l_init_x+l_bar_length, square_size)
    l_pos_y = np.arange(l_init_y, l_init_y+l_bar_height, square_size)
    l_pos_grid = np.meshgrid(l_pos_x, l_pos_y)
    l_pos_xx = l_pos_grid[0].flatten()
    l_pos_yy = l_pos_grid[1].flatten()
    
    for i in range(len(l_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), l_pos_xx[i], l_pos_yy[i], square_size))
        
    # bottom bar
    
    b_bar_length = dim_x
    b_bar_height = int(dim_y*0.2)
    b_init_x = centre_x-(dim_x/2)
    b_init_y = l_pos_y.max() + square_size
    b_pos_x = np.arange(b_init_x, b_init_x+b_bar_length, square_size)
    b_pos_y = np.arange(b_init_y, b_init_y+b_bar_height, square_size)
    b_pos_grid = np.meshgrid(b_pos_x, b_pos_y)
    b_pos_xx = b_pos_grid[0].flatten()
    b_pos_yy = b_pos_grid[1].flatten()
    
    for i in range(len(b_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), b_pos_xx[i], b_pos_yy[i], square_size))
    
        
    return square_list
    
def drawO(centre_x, centre_y, dim_x, dim_y, square_size):
    
    while int(dim_x*0.6)%square_size != 0 or int(dim_y*0.4)%square_size != 0:
        square_size += 1
        
    square_list = []
    
    # top bar
    
    t_bar_length = int(dim_x*0.8)
    t_bar_height = int(dim_y*0.2)
    t_init_x = centre_x-(dim_x/2)
    t_init_y = centre_y-(dim_y/2)+(dim_y*0.2)
    t_pos_x = np.arange(t_init_x, t_init_x+t_bar_length, square_size)
    t_pos_y = np.arange(t_init_y, t_init_y+t_bar_height, square_size)
    t_pos_grid = np.meshgrid(t_pos_x, t_pos_y)
    t_pos_xx = t_pos_grid[0].flatten()
    t_pos_yy = t_pos_grid[1].flatten()
    
    for i in range(len(t_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), t_pos_xx[i], t_pos_yy[i], square_size))
    
    # left bar
    
    l_bar_length = int(dim_x*0.2)
    l_bar_height = int(dim_y*0.6)
    l_init_x = t_init_x
    l_init_y = t_pos_y.max() + square_size
    l_pos_x = np.arange(l_init_x, l_init_x+l_bar_length, square_size)
    l_pos_y = np.arange(l_init_y, l_init_y+l_bar_height, square_size)
    l_pos_grid = np.meshgrid(l_pos_x, l_pos_y)
    l_pos_xx = l_pos_grid[0].flatten()
    l_pos_yy = l_pos_grid[1].flatten()
    
    for i in range(len(l_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), l_pos_xx[i], l_pos_yy[i], square_size))
        
    # bottom bar
    
    b_bar_length = dim_x
    b_bar_height = int(dim_y*0.2)
    b_init_x = centre_x-(dim_x/2)
    b_init_y = l_pos_y.max() + square_size
    b_pos_x = np.arange(b_init_x, b_init_x+b_bar_length, square_size)
    b_pos_y = np.arange(b_init_y, b_init_y+b_bar_height, square_size)
    b_pos_grid = np.meshgrid(b_pos_x, b_pos_y)
    b_pos_xx = b_pos_grid[0].flatten()
    b_pos_yy = b_pos_grid[1].flatten()
    
    for i in range(len(b_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), b_pos_xx[i], b_pos_yy[i], square_size))
        
    # right bar
    
    r_bar_length = int(dim_x*0.2)
    r_bar_height = int(dim_y*0.8)
    r_init_x = t_pos_x.max() + square_size
    r_init_y = centre_y-(dim_y/2)+(dim_y*0.2)
    r_pos_x = np.arange(r_init_x, r_init_x+r_bar_length, square_size)
    r_pos_y = np.arange(r_init_y, r_init_y+r_bar_height, square_size)
    r_pos_grid = np.meshgrid(r_pos_x, r_pos_y)
    r_pos_xx = r_pos_grid[0].flatten()
    r_pos_yy = r_pos_grid[1].flatten()
    
    for i in range(len(r_pos_xx)):
        
        square_list.append(Square(np.random.randint(256, size=(3)), r_pos_xx[i], r_pos_yy[i], square_size))
        
    return square_list

In [ ]:
# functions for audio

def generate_tone(freq, dur_ms, sr):

    A = 0.15
    t = np.arange(0, dur_ms/1000, 1/sr)
    sine = A*np.sin(2*np.pi*freq*t)

    return sine

def save_speak(name):
    synthesizer = pyttsx3.init()
    synthesizer.setProperty('voice', 'com.apple.speech.synthesis.voice.fiona')
    synthesizer.setProperty('rate', 87)

    synthesizer.save_to_file(name+','+','+ 'Welcome to' + ',' + 'C'+','+'2'+','+'H'+','+'O', 'welcome.wav')
    synthesizer.runAndWait() 

    welcome, _ = librosa.load('welcome.wav', sr=sr)
    synthesizer.stop()

    return welcome
    
def amp_envelope(dur_ms, sr=sr):
    
    t = np.arange(0, dur_ms/1000, 1/sr)
    
    # Splitting t up into 4 parts
    attack_length = int(len(t) / 20)
    decay_length = int(len(t) / 4)
    sustain_length = int(len(t) / 2)
    release_length = int(len(t) - (attack_length + decay_length + sustain_length))
    
    # Setting the start and stop amplitube values for the ramps
    attack = np.linspace(0, 0.9, num=attack_length)
    decay = np.linspace(0.9, 0.7, num=decay_length)
    sustain = np.linspace(0.7, 0.5, num=sustain_length)
    release = np.linspace(0.5, 0, num=release_length)
    
    # Putting them together in order
    env = np.concatenate((attack, decay, sustain, release))
    
    return env

def IIRReverb(audio, combq, delay1, gain1, delay2, gain2):

    """A Schroeder reverb with four parallell comb filters and two cascading allpass filters.
    audio = the audio that you will apply the reverb on
    combq = Q factor of the comb filters. Play around but 20 is a good place to start
    delay1 = the delay of the first allpass filter, try settings around 1000
    gain1 = the gain of the first allpass, try anything between 0 and 1, closer to 1
    results in more volume in the reverb
    delay2 = same as delay1 but for the second allpass filter. Try a different and slightly longer
    value than delay1, for example 3500
    gain2 = same as gain1 but for the second allpass filter"""

    s_tail = audio
    s_tail = np.append(s_tail, np.zeros(24000))

    # First comb filter
    f0 = 200
    Q = combq
    b1,a1 = signal.iircomb(f0, Q, ftype='notch', fs=sr)
    y1 = signal.lfilter(b1,a1,s_tail)
    # Second comb filter
    f0 = 300
    Q = combq
    b2,a2 = signal.iircomb(f0, Q, ftype='notch', fs=sr)
    y2 = signal.lfilter(b2,a2,s_tail)
    # Third comb filter
    f0 = 400
    Q = combq
    b3,a3 = signal.iircomb(f0, Q, ftype='notch', fs=sr)
    y3 = signal.lfilter(b3,a3,s_tail)
    # Fourth comb filter
    f0 = 500
    Q = combq
    b4,a4 = signal.iircomb(f0, Q, ftype='notch', fs=sr)
    y4 = signal.lfilter(b4,a4,s_tail)
    # computing up the parallell structure by adding them together (and scaling the amplitude)
    comb_result = (y1 + y2 + y3 + y4)*0.2
    # Designing the two allpass filters
    b1,a1 = allpass(delay1, gain1)
    b2,a2 = allpass(delay2, gain2)

    # Convolving the coefficients to get the cascading structure
    cascade_a = np.convolve(a1,a2)
    cascade_b = np.convolve(b1,b2)

    # Run the result from the comb parallell filters through the cascade of allpass filters
    signal_output = signal.lfilter(cascade_b, cascade_a, comb_result)
    # Merging wet with dry
    #audio = np.append(audio, np.zeros([signal_output.size-audio.size]))
    #output = ((signal_output * wet)+audio)*0.5


    return signal_output

def softClipper(audio, drive, output=0.8):

    """audio = Source to be SoftClipped
    drive = Amount of SoftClipping (Try between 10-40)
    output = Output volume

    The signal is normalized before output attenuation for better control.
    """

    # Drive can not be set to 0
    if drive == 0:
        drive = 1

    piDivisor = 2 / np.pi
    driver = np.arctan(audio * drive)

    softclip = piDivisor * driver

    #normalized = softclip/np.max(softclip)
    softClipped = softclip * output

    return softClipped

def IRDelay(audio, impulses, sr=sr):
    s = np.pad(audio, [0, audio.size])
    s = fade(s, 1024)

    impulse_array = np.linspace(0.2, 0, num=impulses)
    IRarray = np.array([])
    size = int(s.size/impulses)-1
    for i in range(impulses):

        IRarray = np.append(IRarray, np.zeros(size))
        IRarray = np.append(IRarray, impulse_array[i])

    IRarray = sizecheck(IRarray, s)

# transform to frequency domain
    Y = scipy.fft.fft(s)
    H = scipy.fft.fft(IRarray)

# Doing the convolution in the frequency domain
    output = Y*H

# back to time domain
    output = np.real(scipy.fft.ifft(output))
    return output

# Function to add fadeins and fadeouts to the segments to avoid clicks
def fade(segment, fadesize):
    fades = np.array([])
    fadesize = fadesize
    if segment.size <= (fadesize*2):
        fadesize = int(segment.size / 10)
        midpart = np.ones(segment.size - (int(fadesize*2)))
        fadein = np.linspace(0,1,num=fadesize)
        fadeout = np.linspace(1,0,num=fadesize)
        fades = np.concatenate((fadein, midpart, fadeout))
        
    else:         
        midpart = np.ones(segment.size - (int(fadesize*2)))   
        fadein = np.linspace(0,1,num=fadesize)
        fadeout = np.linspace(1,0,num=fadesize)
        fades = np.concatenate((fadein, midpart, fadeout))
        
    y = segment * fades
    
    return y

# Function to check if a segment has the correct size, and if not to make sure it is right
# by padding or deleting samples. 
def sizecheck(segment, result):
    if segment.size != result.size:
        if segment.size > result.size:
            diff = segment.size-result.size            
            segment = np.delete(segment, diff)
            
        elif result.size > segment.size:
            zeros= np.zeros(result.size - segment.size)
            
            segment = np.concatenate((segment, zeros))
    else:
        pass
    
    return segment

def allpass(delay, gain):
    b = np.zeros(delay)
    b[0] = gain
    b[delay-1] = 1
    a = np.zeros(delay)
    a[0] = 1
    a[delay-1] = gain
    return b,a

In [ ]:
# Initialising the audio in arrays

# Dictionary of digits and frequencies

DTFT_dials = {
    1 : [697,1209], # 1
    2 : [697,1336], # 2
    3 : [697,1477], # 3
    4 : [770,1209], # 4  
    5 : [770,1336], # 5
    6 : [770,1477], # 6
    7 : [852,1209], # 7
    8 : [852,1336], # 8 
    9 : [852,1477], # 9
    '*' : [941,1209], # *
    0 : [941,1336], # 0
    '#' : [941,1477] # #
}

dial_tone = generate_tone(350, 1000, sr) + generate_tone(440, 1000, sr)
dial_tone = dial_tone*100000
dial_tone = dial_tone.astype(np.int16)

dial_sequence = []

for i in ip:
    freqs = DTFT_dials.get(i)
    s = generate_tone(freqs[0], 125, sr)
    s1 = generate_tone(freqs[1], 125, sr)
    s = s + s1
    s = np.concatenate((s, np.zeros(50)))
    s = s*100000
    s = s.astype(np.int16)
    dial_sequence.append(s)
    
short = 50
long = 120
pause = 600
seq = [long, short, long, short, pause, short, short, long, long, long, pause, short, short, short, short, pause, long, long, long, pause, long, short, long, short, pause, short, short, long, long, long, pause, short, short, short, short, pause, long, long, long, pause]

counter = 0
# Creating the dial tone
dialfun = np.array([])

for i in range(len(seq)):

    if counter >= len(ip):
        break
    elif seq[i] != pause:
        freqs = DTFT_dials.get(ip[counter])
        s = generate_tone(freqs[0], seq[i], sr)
        env = amp_envelope(seq[i], sr=sr)
        s = s * env
        s1 = generate_tone(freqs[1], seq[i], sr)
        env = amp_envelope(seq[i], sr=sr)
        s1 = s1 * env
        s = s + s1
        dialfun = np.append(dialfun, s)
        dialfun = np.append(dialfun, np.zeros(1200))
    elif seq[i]== pause:
        s = np.zeros(int((pause/1000)*sr))
        counter += 1
        dialfun = np.append(dialfun, s)

# Adding FX (rev and delay, original dry signal is kept as well)
dialfun = np.pad(dialfun, (0,40000))
delay = IRDelay(dialfun, 7)
rev = IIRReverb(delay, 20, 1000, 0.5, 2600, 0.3)
dialfun = np.pad(dialfun, (0,rev.size-dialfun.size))
total = rev + dialfun*0.6
# Adding softclipper to total signal
total = softClipper(total, 5, 0.6)

       
total = total*100000
dialfun = total.astype(np.int16)

welcome = save_speak(name)
welcome = welcome*10000
welcome = welcome.astype(np.int16)

In [ ]:
# if running on mac M1 ()
# !pip install git+https://github.com/Muxelmann/pygame.git@patch-1 

In [ ]:
# initialise the paddles for pong

paddle_list = [Paddle(white, 10, height/2, 10, int(height/10)),
              Paddle(white, width-30, height/2, 10, int(height/10))]

In [ ]:
# function for pyaudio stream for render

def audio(extract):

    # read samples for frame from stream to buffer

    data = stream.read(chunk)

    # convert to float

    data = np.frombuffer(data, dtype=np.int16)

    audio_list.append(data)

    return(data)

The following cell is the main programme flow.

In [ ]:
# Main Pygame loop

# pygame stream for recording audio for stream

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    device_dict = p.get_device_info_by_index(i)
    if 'Stereo Mix' in device_dict['name']:
        device_index = device_dict['index']
        break
        
chunk = 1024        
stream = p.open(format=pyaudio.paInt16,
                  channels=1,
                  rate=sr,
                  frames_per_buffer=chunk,
                  input_device_index = device_index,
                  input=True)

pool = ThreadPool(processes=1)

# initalise and run pygame

pygame.mixer.pre_init(frequency=sr,size=-16, buffer=2048, channels=1, allowedchanges=0)
pygame.init()
pygame.mixer.init(frequency=sr, size=-16, channels=1, buffer=2048)
clock = pygame.time.Clock()
screen = pygame.display.set_mode((width, height))

h_centre_x = width/2
h_centre_y = height/2
h_square_list = drawH(h_centre_x, h_centre_y, 80, 120, 6)

c_centre_x = (width/2)-(width/4)
c_centre_y = (height/2)-(height/4)
c_square_list = drawC(c_centre_x, c_centre_y, 80, 120, 6)

t_centre_x = (width/2) + (width/4)
t_centre_y = (height/2) - (height/4)
t_square_list = draw2(t_centre_x, t_centre_y, 80, 120, 6)

o_centre_x = width/2
o_centre_y = (height/2) + (height/4)
o_square_list = drawO(o_centre_x, o_centre_y, 80, 120, 6)

square_dump = h_square_list + c_square_list + t_square_list + o_square_list
random.shuffle(square_dump)

square_list = []
square_list.append(square_dump[0])

max_x = max(square.pos_x for square in square_dump)
min_x = min(square.pos_x for square in square_dump)
max_y = max(square.pos_y for square in square_dump)
min_y = min(square.pos_y for square in square_dump)

all_rendered = 0

next_pong = 0
pong_released = 0

pong_mover_x = 2
pong_mover_y = 2

frame_count = 0

square_counter = 1

dial_counter = 0

sound_playing = 0

dialfun_playcount = 0

video_recording_list = []
audio_list = []

# run main loop

while True:
    
    async_result = pool.apply_async(audio, (chunk,))
    
    # fill background
    
    screen.fill(screen_fill)
    
    sound_playing = pygame.mixer.get_busy()
    
    # escape key to exit
    
    for event in pygame.event.get():
        
        if event.type == pygame.KEYDOWN:
            
            if event.key == pygame.K_ESCAPE:
                
                pygame.mixer.quit()
                pygame.quit()
                sys.exit()
                
    if frame_count%1 == 0 and frame_count > fps*6:
        
        if square_counter >= len(square_dump):
            
            pass
        
        else:
        
            square_list.append(square_dump[square_counter])
            square_counter += 1
            
            if square_counter == len(square_dump)-1:
                # Then do something here to make it play
                welcome_sound = pygame.sndarray.make_sound(welcome)
                welcome_sound.play()
                all_rendered = 1
                next_pong = 1
            
    if all_rendered == 1 and next_pong == 1:
        
        next_pong = 0
        
        x_min_max = np.random.randint(2)
        
        if x_min_max == 0:
            
            square_release_column = min_x
            
        if x_min_max == 1:
            
            square_release_column = max_x
                
        pong_index = next(index for index, square in enumerate(square_list) if square.pos_x == square_release_column)
        pong_square = next(square for index, square in enumerate(square_list) if square.pos_x == square_release_column)
        
        pong_released = 1
        
        pong_mover_y = np.random.choice([-1,1])
        
    if pong_released == 1:
        
        if pong_square.pos_y < 0 or pong_square.pos_y > height:
            pong_mover_y = -pong_mover_y
        
        if x_min_max == 0:
            
            pong_square.pos_x = pong_square.pos_x - pong_mover_x
            pong_square.pos_y = pong_square.pos_y - pong_mover_y
            
        if x_min_max == 1:
            
            pong_square.pos_x = pong_square.pos_x + pong_mover_x
            pong_square.pos_y = pong_square.pos_y + pong_mover_y
            
        if pong_square.pos_x < 0 or pong_square.pos_x > width:   
        
            if len(square_list) != 1:
        
                del square_list[pong_index]
            
                max_x = max(square.pos_x for square in square_list)
                min_x = min(square.pos_x for square in square_list)

                next_pong = 1
                
        keys = pygame.key.get_pressed()
                
        if keys[K_UP]:
            
            if paddle_list[1].pos_y <= 0:
                
                pass
            
            else:
                
                paddle_list[1].pos_y -= 2

        if keys[K_DOWN]:
            
            if paddle_list[1].pos_y + paddle_list[1].height >= height:
                
                pass
            
            else:

                paddle_list[1].pos_y += 2

        if keys[K_w]:
            
            if paddle_list[0].pos_y <= 0:
                
                pass
            
            else:

                paddle_list[0].pos_y -= 2

        if keys[K_s]:
            
            if paddle_list[0].pos_y+paddle_list[0].height >= height:
                
                pass
            
            else:

                paddle_list[0].pos_y += 2

        if square_list[pong_index].pos_x < (paddle_list[0].pos_x + paddle_list[0].width) and square_list[pong_index].pos_y > paddle_list[0].pos_y and square_list[pong_index].pos_y < (paddle_list[0].pos_y+paddle_list[0].height):
            pong_mover_x = -pong_mover_x
            
        if square_list[pong_index].pos_x > paddle_list[1].pos_x and square_list[pong_index].pos_y > paddle_list[1].pos_y and square_list[pong_index].pos_y < (paddle_list[1].pos_y+paddle_list[1].height):
            pong_mover_x = -pong_mover_x
                


    if frame_count%fps*1.5 == 0 and frame_count < fps*6 and sound_playing == 0:
        dial_sound =  pygame.sndarray.make_sound(dial_tone)
        dial_sound.play()
        
    if frame_count > fps*6 and dial_counter < len(dial_sequence) and sound_playing == 0 and frame_count%(fps/4)==0:
        
        ip_tone = pygame.sndarray.make_sound(dial_sequence[dial_counter])
        ip_tone.play()
        dial_counter += 1
        if dial_counter == len(dial_sequence)-1:
            dial_wait_frame = frame_count
        
    if dial_counter >= len(ip) and sound_playing == 0 and dialfun_playcount == 0 and frame_count == dial_wait_frame + fps*0.5:
        
        dialfun_sound = pygame.sndarray.make_sound(dialfun)
        dialfun_sound.play()
        dialfun_playcount = 1
        
    if frame_count > fps*6:
        
        for square in square_list:

            square.draw(screen)
            
    if pong_released == 1:
        
        for paddle in paddle_list:
            
            paddle.draw(screen)
                
    # refresh display
    clock.tick_busy_loop(60)
    cap_frame = pygame.surfarray.array3d(screen)
    video_recording_list.append(cap_frame)
    pygame.display.flip()
    frame_count += 1

# Render

## Audio

In [ ]:
audio_array = np.array(audio_list)
audio_array = np.concatenate(audio_array)
audio_array = np.array(audio_array, dtype=np.float32)
audio_array = audio_array/audio_array.max()

In [ ]:
ffmpegio.audio.write('audio.wav', sr, audio_array)

## Video

In [ ]:
video_array = np.array(video_recording_list)

vid_out = skvideo.io.FFmpegWriter('Render.mp4', inputdict={
        '-r': '60',
      },
      outputdict={
        '-vcodec': 'libx264',
        '-pix_fmt': 'yuv444p',
        '-r': '60',
        '-crf': '17',
  })

for frame in video_array:
    frame = np.fliplr(scipy.ndimage.rotate(frame, 270))
    vid_out.writeFrame(frame)

vid_out.close()

In [ ]:
# from https://stackoverflow.com/questions/56973205/how-to-combine-the-video-and-audio-files-in-ffmpeg-python

input_video = ffmpeg.input('Render.mp4')

input_audio = ffmpeg.input('audio.wav')

ffmpeg.concat(input_video, input_audio, v=1, a=1).output('RenderwithSound.mp4').run()
